# Sentiment Analyses using a word embedding layer and a transformer

Zielsetzung:

Alle Imports:

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from tensorflow import keras
from tensorflow.keras import layers


# Authenticating with Kaggle using kaggle.json

Navigate to https://www.kaggle.com. Then go to the [Account tab of your user profile](https://www.kaggle.com/me/account) and select Create API Token. This will trigger the download of kaggle.json, a file containing your API credentials.

Then run the cell below to upload kaggle.json to your Colab runtime.

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 62 bytes


In [3]:
!kaggle datasets download -d nelgiriyewithana/mcdonalds-store-reviews
!unzip mcdonalds-store-reviews.zip

  0% 0.00/1.78M [00:00<?, ?B/s]
100% 1.78M/1.78M [00:00<00:00, 175MB/s]
Archive:  mcdonalds-store-reviews.zip
  inflating: McDonald_s_Reviews.csv  


In [4]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
!unzip imdb-dataset-of-50k-movie-reviews.zip

100% 25.7M/25.7M [00:00<00:00, 140MB/s] 
100% 25.7M/25.7M [00:00<00:00, 125MB/s]
Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [17]:
df_mc = pd.read_csv('McDonald_s_Reviews.csv', encoding="latin-1")
df_imdb = pd.read_csv('IMDB Dataset.csv')
df_mc = df_mc[df_mc['rating'] != '3 stars']
data_mc = df_mc['review'].to_numpy()
data_imdb = df_imdb['review'].to_numpy()
rating_mapping_imdb = {
    'positive': 1,
    'negative': 0,
}

label_imdb = df_imdb['sentiment'].map(rating_mapping_imdb).to_numpy()
rating_mapping_mc = {
    '1 star': 0,
    '2 stars': 0,
    '4 stars': 1,
    '5 stars': 1
}

label_mc = df_mc['rating'].map(rating_mapping_mc).to_numpy()
data = np.append(data_imdb, data_mc)
label = np.append(label_imdb,label_mc)

train_data, test_data, train_label, test_label = train_test_split(data, label, test_size=0.2, random_state=42)




In [18]:
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(train_data)
text_vectorization.adapt(test_data)

int_train_ds = text_vectorization(train_data)

int_test_ds = text_vectorization(test_data)

In [19]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [20]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_4 (Embedding)     (None, None, 256)         5120000   
                                                                 
 transformer_encoder_3 (Tran  (None, None, 256)        543776    
 sformerEncoder)                                                 
                                                                 
 global_max_pooling1d_3 (Glo  (None, 256)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_11 (Dense)            (None, 1)                 257 

In [23]:
if None in int_train_ds:
    print("The array contains None values.")
else:
    print("The array does not contain None values.")

The array does not contain None values.


In [10]:
callbacks = [
  keras.callbacks.EarlyStopping(
        monitor="val_loss",  # Metric to monitor
        patience=3,  # Number of epochs with no improvement after which training will be stopped
        restore_best_weights=True,  # Restore the weights of the best epoch
    )
]
model.fit(int_train_ds, train_label, validation_split=0.4, epochs=20, callbacks=callbacks)

Epoch 1/20
1179/1179 [==============================] - 168s 139ms/step - loss: 0.4232 - accuracy: 0.8064 - val_loss: 0.3246 - val_accuracy: 0.8597
Epoch 2/20
1179/1179 [==============================] - 109s 93ms/step - loss: 0.3204 - accuracy: 0.8619 - val_loss: 0.2985 - val_accuracy: 0.8719
Epoch 3/20
1179/1179 [==============================] - 100s 85ms/step - loss: 0.2928 - accuracy: 0.8762 - val_loss: 0.2887 - val_accuracy: 0.8774
Epoch 4/20
1179/1179 [==============================] - 97s 82ms/step - loss: 0.2734 - accuracy: 0.8853 - val_loss: 0.2820 - val_accuracy: 0.8781
Epoch 5/20
1179/1179 [==============================] - 97s 82ms/step - loss: 0.2595 - accuracy: 0.8926 - val_loss: 0.2814 - val_accuracy: 0.8807
Epoch 6/20
1179/1179 [==============================] - 95s 80ms/step - loss: 0.2421 - accuracy: 0.9000 - val_loss: 0.2827 - val_accuracy: 0.8819
Epoch 7/20
1179/1179 [==============================] - 96s 81ms/step - loss: 0.2283 - accuracy: 0.9082 - val_loss: 0.30

In [11]:
print(model.evaluate(int_test_ds, test_label))

492/492 [==============================] - 12s 24ms/step - loss: 0.2717 - accuracy: 0.8862
[0.2716885209083557, 0.8861669898033142]


In [21]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [22]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
  keras.callbacks.EarlyStopping(
        monitor="val_loss",  # Metric to monitor
        patience=3,  # Number of epochs with no improvement after which training will be stopped
        restore_best_weights=True,  # Restore the weights of the best epoch
    )
]


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding_1 (Pos  (None, None, 256)        5273600   
 itionalEmbedding)                                               
                                                                 
 transformer_encoder_4 (Tran  (None, None, 256)        543776    
 sformerEncoder)                                                 
                                                                 
 global_max_pooling1d_4 (Glo  (None, 256)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                           

In [23]:
model.fit(int_train_ds,train_label, validation_split=0.4, epochs=20, callbacks=callbacks)


Epoch 1/20
1179/1179 [==============================] - 147s 122ms/step - loss: 0.4139 - accuracy: 0.8118 - val_loss: 0.4414 - val_accuracy: 0.8140
Epoch 2/20
1179/1179 [==============================] - 132s 112ms/step - loss: 0.2694 - accuracy: 0.8894 - val_loss: 0.2934 - val_accuracy: 0.8744
Epoch 3/20
1179/1179 [==============================] - 125s 106ms/step - loss: 0.2330 - accuracy: 0.9068 - val_loss: 0.2643 - val_accuracy: 0.8887
Epoch 4/20
1179/1179 [==============================] - 124s 105ms/step - loss: 0.2069 - accuracy: 0.9182 - val_loss: 0.2878 - val_accuracy: 0.8906
Epoch 5/20
1179/1179 [==============================] - 104s 88ms/step - loss: 0.1855 - accuracy: 0.9288 - val_loss: 0.2899 - val_accuracy: 0.8873
Epoch 6/20
1179/1179 [==============================] - 121s 103ms/step - loss: 0.1654 - accuracy: 0.9391 - val_loss: 0.2763 - val_accuracy: 0.8890


In [24]:
model.evaluate(int_test_ds, test_label)

492/492 [==============================] - 14s 29ms/step - loss: 0.2517 - accuracy: 0.8953


[0.25168943405151367, 0.8952659964561462]